분류 성능 평가
- 카테고리를 4개로 나눴을 때

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
%cd /content/drive/MyDrive/my_pjt(주간)

/content/drive/MyDrive/my_pjt(주간)


In [31]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [32]:
df_row = pd.read_csv('음식_goodpoint.csv', index_col=0)
df = df_row.copy()
df.head()

,Attraction,Good_Point,Selected_People
803,채그로,뷰가 좋아요,146
804,채그로,집중하기 좋아요,62
805,채그로,읽을만한 책이 많아요,61
806,채그로,인테리어가 멋져요,60
807,채그로,커피가 맛있어요,39


In [33]:
category_map = {
    "시설 및 분위기": ["뷰가 좋아요", "인테리어가 멋져요", "사진이 잘 나와요", "경관이 독특해요",
                    "좌석이 편해요", "매장이 넓어요", "룸이 잘 되어있어요", "야외 공간이 멋져요",
                    "산책로가 잘 되어있어요", "피크닉하기 좋아요", "편의시설이 잘 되어있어요", "공간이 넓어요",
                    "붐비지 않아요", "방문객이 많아요",
                    "특별한 날 가기 좋아요", "컨셉이 독특해요", "테마가 재미있어요", "아늑해요",
                    "차분한 분위기에요", "오래 머무르기 좋아요", "조용히 쉬기 좋아요", "혼밥하기 좋아요", "혼술하기 좋아요",
                    "파티하기 좋아요", "라이브공연이 훌륭해요", "음악이 좋아요", "대화하기 좋아요", "집중하기 좋아요",
                    "화장실이 깨끗해요", "매장이 청결해요", "환기가 잘 돼요", "관리가 잘 되어있어요",
                    "단체모임 하기 좋아요", "아이와 가기 좋아요", "부모도 쉬기 좋아요", "읽을만한 책이 많아요",
                    "볼거리가 많아요", "반려동물과 가기 좋아요", "선물하기 좋아요",
                    "놀거리가 많아요", "체험 프로그램이 많아요"],

    "음식의 속성": ["커피가 맛있어요", "음료가 맛있어요", "디저트가 맛있어요", "차가 맛있어요",
                    "재료가 신선해요", "건강한 맛이에요", "신선해요", "원산지 표시가 명확해요",
                    "음식이 맛있어요", "양이 많아요", "고기 질이 좋아요", "잡내가 적어요", "빵이 맛있어요", "기본 안주가 좋아요",
                    "특별한 메뉴가 있어요", "반찬이 잘 나와요", "술이 다양해요",
                    "코스가 길어요", "먹거리가 풍부해요", "샐러드바가 잘 되어있어요", "부위가 다양해요", "종류가 다양해요",
                    "향신료가 강하지 않아요","현지 맛에 가까워요", "비주얼이 좋아요"],

    "가격 및 가치": ["가성비가 좋아요", "코스요리가 알차요", "세트 구성이 알차요", "비싼 만큼 가치있어요",
                     "가격이 합리적이에요", "메뉴 구성이 알차요"],

    "서비스" : ["친절해요", "음식이 빨리 나와요", "포장이 깔끔해요", "직접 잘 구워줘요", "주문제작을 잘해줘요", "안전하게 관리해요"],


    "주변 인프라": ["대중교통이 편해요", "근처에 갈 곳이 많아요", "주차하기 편해요" ],

}

In [34]:
# df에 속성을 업로
def update_categories(df, category_map):
    for category, keywords in category_map.items():
        for keyword in keywords:
            df.loc[df['Good_Point'].str.contains(keyword), 'Category_Map'] = category
    return df

df = update_categories(df, category_map)

In [35]:
df.head()

,Attraction,Good_Point,Selected_People,Category_Map
803,채그로,뷰가 좋아요,146,시설 및 분위기
804,채그로,집중하기 좋아요,62,시설 및 분위기
805,채그로,읽을만한 책이 많아요,61,시설 및 분위기
806,채그로,인테리어가 멋져요,60,시설 및 분위기
807,채그로,커피가 맛있어요,39,음식의 속성


# 명사와 형용사로 토큰화하여 분류하기

In [13]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.8 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=428463d3310e0d360bec509012e30538ee48084c2108983c597e85e60677a9bf
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1984k      0 --:--:-- --:--:-- --:--:-- 1984k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example

In [36]:
import json
import re
from konlpy.tag import Okt
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import font_manager, rc
from wordcloud import WordCloud

from konlpy.tag import Okt

In [37]:
# 카테고리화 한 것을 데이터프레임으로 변경
category_list =[]
Good_Point_list = []

def classification_df(category_map):
    for category, keywords in category_map.items():

        for keyword in keywords:
            category_list.append(category)
            Good_Point_list.append(keyword)

    data ={'category' : category_list, 'Good_Point' : Good_Point_list}
    for_word_tocken = pd.DataFrame(data)
    return for_word_tocken

for_word_tocken = classification_df(category_map)
for_word_tocken

,category,Good_Point
0,시설 및 분위기,뷰가 좋아요
1,시설 및 분위기,인테리어가 멋져요
2,시설 및 분위기,사진이 잘 나와요
3,시설 및 분위기,경관이 독특해요
4,시설 및 분위기,좌석이 편해요
...,...,...
76,서비스,주문제작을 잘해줘요
77,서비스,안전하게 관리해요
78,주변 인프라,대중교통이 편해요
79,주변 인프라,근처에 갈 곳이 많아요


In [38]:
# 각 리뷰를 형용사와 명사로 구분하기

word = []

for Good_Point in for_word_tocken['Good_Point']:
#   print(item)
    #if 'message' in item:
        new_message = re.sub(r'[^\w]',' ', Good_Point)
        # Python의 re 모듈에서 제공하는 sub 함수는 문자열에서 패턴에 일치하는 부분을 다른 문자열로 대체
        # item['message']의 모든 비단어 문자를 공백으로 대체. 여기서 비단어 문자는 알파벳, 숫자, 밑줄(_)을 제외한 모든 문자
        # [^]: 대괄호 안의 ^는 부정을 의미합니다. 즉, 대괄호 안의 내용과 일치하지 않는 문자를 찾습니다.
        # \w: 단어 문자를 의미합니다. 이는 알파벳 대소문자(a-z, A-Z), 숫자(0-9), 밑줄(_)을 포함합니다.
        # 종합하면, r'[^\w]'는 "단어 문자가 아닌 모든 문자"를 찾는 정규 표현식입니다.

        nlp = Okt()
        pos_tags = nlp.pos(new_message)
        nouns_and_adjectives = [word for word, tag in pos_tags if tag in ['Noun', 'Adjective']]
        word.append(nouns_and_adjectives)

# data =
for_word_tocken['word'] = pd.DataFrame({'word':word})
for_word_tocken

,category,Good_Point,word
0,시설 및 분위기,뷰가 좋아요,"[뷰, 좋아요]"
1,시설 및 분위기,인테리어가 멋져요,"[인테리어, 멋져요]"
2,시설 및 분위기,사진이 잘 나와요,[사진]
3,시설 및 분위기,경관이 독특해요,"[경관, 독특해요]"
4,시설 및 분위기,좌석이 편해요,"[좌석, 편해요]"
...,...,...,...
76,서비스,주문제작을 잘해줘요,[제작]
77,서비스,안전하게 관리해요,"[안전하게, 관리]"
78,주변 인프라,대중교통이 편해요,"[대중교통, 편해요]"
79,주변 인프라,근처에 갈 곳이 많아요,"[근처, 곳, 많아요]"


In [39]:
for_word_tocken['word'] = for_word_tocken['word'].apply(lambda x: ', '.join(map(str, x)))

for_word_tocken

,category,Good_Point,word
0,시설 및 분위기,뷰가 좋아요,"뷰, 좋아요"
1,시설 및 분위기,인테리어가 멋져요,"인테리어, 멋져요"
2,시설 및 분위기,사진이 잘 나와요,사진
3,시설 및 분위기,경관이 독특해요,"경관, 독특해요"
4,시설 및 분위기,좌석이 편해요,"좌석, 편해요"
...,...,...,...
76,서비스,주문제작을 잘해줘요,제작
77,서비스,안전하게 관리해요,"안전하게, 관리"
78,주변 인프라,대중교통이 편해요,"대중교통, 편해요"
79,주변 인프라,근처에 갈 곳이 많아요,"근처, 곳, 많아요"


In [40]:
for_word_tocken[for_word_tocken['word'] == ""]

,category,Good_Point,word
12,시설 및 분위기,붐비지 않아요,
52,음식의 속성,잡내가 적어요,


In [41]:
# 명사나 형용사로 나누어지지 않은 단어 채워주기
for_word_tocken.iloc[12,2] = '붐비지'
for_word_tocken.iloc[14,2] = '특별한 날'
for_word_tocken.iloc[36,2] = '볼거리'
for_word_tocken.iloc[39,2] = '놀거리'
for_word_tocken.iloc[52,2] = '잡내'
for_word_tocken.iloc[60,2] = '특별한 메뉴'
for_word_tocken.iloc[55,2] = '가성비'
for_word_tocken.iloc[72,2] = '가성비' # 가성, 비
for_word_tocken.iloc[72,2] = '비주얼' # 비주, 얼

In [42]:
# 명사와 령용사로 최종 구분 : for_word_tocken
for_word_tocken

,category,Good_Point,word
0,시설 및 분위기,뷰가 좋아요,"뷰, 좋아요"
1,시설 및 분위기,인테리어가 멋져요,"인테리어, 멋져요"
2,시설 및 분위기,사진이 잘 나와요,사진
3,시설 및 분위기,경관이 독특해요,"경관, 독특해요"
4,시설 및 분위기,좌석이 편해요,"좌석, 편해요"
...,...,...,...
76,서비스,주문제작을 잘해줘요,제작
77,서비스,안전하게 관리해요,"안전하게, 관리"
78,주변 인프라,대중교통이 편해요,"대중교통, 편해요"
79,주변 인프라,근처에 갈 곳이 많아요,"근처, 곳, 많아요"


In [43]:
# 분류를 위한 리스트 변경 : ㅈ word_list
word_list = for_word_tocken['word'].to_list()

In [44]:
word_list = list(set(word_list))
word_list

['비싼, 만큼',
 '반찬',
 '특별한 메뉴',
 '음식',
 '관리',
 '대화, 좋아요',
 '특별한 날',
 '음료, 맛있어요',
 '편의, 시설',
 '매장, 넓어요',
 '공간, 넓어요',
 '재료, 신선해요',
 '책, 많아요',
 '가성비',
 '반려동물, 가기, 좋아요',
 '체험, 프로그램, 많아요',
 '집중, 좋아요',
 '안전하게, 관리',
 '메뉴, 구성, 알차요',
 '산책로',
 '좋아요',
 '환기',
 '테마, 재미있어요',
 '부모, 좋아요',
 '놀거리',
 '좌석, 편해요',
 '부위, 다양해요',
 '현지, 맛, 가까워요',
 '종류, 다양해요',
 '직접',
 '매장, 청결해요',
 '방문객, 많아요',
 '인테리어, 멋져요',
 '룸',
 '차분한, 분위기',
 '향신료, 강하지',
 '비주얼',
 '기본, 안주, 좋아요',
 '비주, 얼, 좋아요',
 '조용히, 좋아요',
 '신선해요',
 '코스',
 '혼술, 좋아요',
 '고기, 질, 좋아요',
 '아늑해요',
 '커피, 맛있어요',
 '경관, 독특해요',
 '근처, 곳, 많아요',
 '아이, 가기, 좋아요',
 '코스, 요리, 알차요',
 '붐비지',
 '잡내',
 '선물, 좋아요',
 '사진',
 '뷰, 좋아요',
 '제작',
 '음식, 맛있어요',
 '볼거리',
 '파티, 좋아요',
 '음악, 좋아요',
 '원산지, 표시, 명확해요',
 '컨셉, 독특해요',
 '대중교통, 편해요',
 '포장, 깔끔해요',
 '야외, 공간, 멋져요',
 '디저트, 맛있어요',
 '먹거리, 풍부해요',
 '화장실, 깨끗해요',
 '차가, 맛있어요',
 '주차, 편해요',
 '빵, 맛있어요',
 '양, 많아요',
 '혼밥, 좋아요',
 '가격, 합리',
 '세트, 구성, 알차요',
 '술, 다양해요',
 '라이브, 공연, 훌륭해요',
 '건강한, 맛',
 '피크닉, 좋아요',
 '가성, 비, 좋아요',
 '단체, 모임, 좋아요']

단어, 형용사 별로 분류

In [45]:
new_category_map = {
    "시설 및 분위기": ["뷰", "인테리어", "멋져요", "사진", "경관",
                    "좌석", "매장", "넓어요", "룸", "야외", "공간",
                    "산책로", "피크닉", "편의시설", "공간","넓어요",
                    "붐비지", "방문객",
                    "특별한 날", "컨셉", "테마", "아늑해요",
                    "차분한", "분위기", "머무르기", "조용히", "혼밥", "혼술",
                    "파티", "라이브", "공연", "음악", "대화하기 좋아요", "집중",
                    "화장실","깨끗해요", "청결해요", "환기", "관리",
                    "단체","모임", "아이", "부모", "책",
                    "볼거리", "반려동물", "선물",
                    "놀거리", "체험", "프로그램"],

    "음식의 속성": ["맛있어요", "음료", "디저트",
                    "재료", "신선해요", "건강한", "신선해요", "원산지",
                    "음식", "양", "고기", "질", "잡내", "빵", "안주",
                    "특별한 메뉴", "반찬이 잘 나와요", "술",
                    "먹거리", "샐러드", "부위", "종류",
                    "향신료","현지 맛", "비주얼"],

    "가격 및 가치": ["가성비", "코스", "구성", "비싼", "가치",
                    "가격"," 합리"],

    "서비스" : ["친절해요", "빨리", "포장", "깔끔", "직접","구워줘요",
                "제작", "안전하게"],


    "주변 인프라": ["대중교통이 편해요", "근처", "주차" ],

}

# 단어를 이용하여 속성 분리
- predicted_category 로 표현

In [47]:
# Pred_Category_Map : 단어로 토큰화 한 것을보 분류한 것
def update_categories(df, category_map):
    for category, keywords in category_map.items():
        for keyword in keywords:
            df.loc[df['Good_Point'].str.contains(keyword), 'Pred_Category_Map'] = category
    return df

df = update_categories(df, new_category_map)
df

,Attraction,Good_Point,Selected_People,Category_Map,Pred_Category_Map
803,채그로,뷰가 좋아요,146,시설 및 분위기,시설 및 분위기
804,채그로,집중하기 좋아요,62,시설 및 분위기,시설 및 분위기
805,채그로,읽을만한 책이 많아요,61,시설 및 분위기,시설 및 분위기
806,채그로,인테리어가 멋져요,60,시설 및 분위기,시설 및 분위기
807,채그로,커피가 맛있어요,39,음식의 속성,음식의 속성
...,...,...,...,...,...
17442,h450,아늑해요,2,시설 및 분위기,시설 및 분위기
17443,h450,메뉴 구성이 알차요,2,가격 및 가치,가격 및 가치
17444,h450,건강한 맛이에요,2,음식의 속성,음식의 속성
17445,h450,음료가 맛있어요,1,음식의 속성,음식의 속성


In [49]:
# 피벗 테이블 생성
Pivot_df = df.pivot_table(
    index='Attraction',
    columns='Category_Map',
    values='Selected_People',
    aggfunc='sum',  # 중복된 경우 합계
    fill_value=0    # NaN 값을 0으로 채움
)

In [50]:
# 결과 출력
Pivot_df

Category_Map,가격 및 가치,서비스,시설 및 분위기,음식의 속성,주변 인프라
Attraction,,,,,
1940금문,6,43,78,210,1
63뷔페 파빌리온,2372,1974,6130,6137,971
BK볏짚 우대갈비 장안동본점,62,285,500,887,51
RSG 성수,0,2,17,13,0
WANGTHAI,31,173,325,567,17
...,...,...,...,...,...
황금정원,172,254,1147,2009,373
황생가칼국수,323,921,1795,5523,96
황토추어탕,224,154,216,750,57


In [51]:
Pivot_df['Total_Feature_Counts'] = Pivot_df.sum(axis=1)

In [52]:
Pivot_df.sort_values(by='Total_Feature_Counts', ascending=False)

Category_Map,가격 및 가치,서비스,시설 및 분위기,음식의 속성,주변 인프라,Total_Feature_Counts
Attraction,,,,,,
명동칼국수 본점,2270,2992,8696,30767,96,44821
태극당,623,1702,6260,15718,49,24352
코코넛박스,268,1045,10392,6318,18,18041
63뷔페 파빌리온,2372,1974,6130,6137,971,17584
런던베이글뮤지엄,401,1348,4262,10115,8,16134
...,...,...,...,...,...,...
RSG 성수,0,2,17,13,0,32
홍어명가,5,4,8,13,0,30
우스마니아,1,5,7,16,0,29


In [53]:
Pivot_df

Category_Map,가격 및 가치,서비스,시설 및 분위기,음식의 속성,주변 인프라,Total_Feature_Counts
Attraction,,,,,,
1940금문,6,43,78,210,1,338
63뷔페 파빌리온,2372,1974,6130,6137,971,17584
BK볏짚 우대갈비 장안동본점,62,285,500,887,51,1785
RSG 성수,0,2,17,13,0,32
WANGTHAI,31,173,325,567,17,1113
...,...,...,...,...,...,...
황금정원,172,254,1147,2009,373,3955
황생가칼국수,323,921,1795,5523,96,8658
황토추어탕,224,154,216,750,57,1401


# 분류 성능 평가

In [59]:
df

,Attraction,Good_Point,Selected_People,Category_Map,Pred_Category_Map
803,채그로,뷰가 좋아요,146,시설 및 분위기,시설 및 분위기
804,채그로,집중하기 좋아요,62,시설 및 분위기,시설 및 분위기
805,채그로,읽을만한 책이 많아요,61,시설 및 분위기,시설 및 분위기
806,채그로,인테리어가 멋져요,60,시설 및 분위기,시설 및 분위기
807,채그로,커피가 맛있어요,39,음식의 속성,음식의 속성
...,...,...,...,...,...
17442,h450,아늑해요,2,시설 및 분위기,시설 및 분위기
17443,h450,메뉴 구성이 알차요,2,가격 및 가치,가격 및 가치
17444,h450,건강한 맛이에요,2,음식의 속성,음식의 속성
17445,h450,음료가 맛있어요,1,음식의 속성,음식의 속성


# 리뷰 자체를 토큰화

## 텍스트를 벡터화
- 위에 기준하에 가격 및 가치,	서비스,	시설 및 분위기,	음식의 속성, 주변 인프라
- 5개로 속성을 나누고, 그에 따라 리뷰를 분류 했을 때,
-  랜덤으로 80%로 위의 기준에 따라 나눈 것을 학습 시키고,
-  나머지 20%로를 훈련

- 정확도가 매우 높은 이유는 20%의 x_train에 해당하는 리뷰가 이미 학습된 상태이기 때문
- 만약 학습시킨 리뷰에 없는 전혀 다른 내용이 들어온다면 학습데이터에 존재하지 않기에 적절히 분류가 어려울 있음



- 텍스트 데이터를 벡터화하면(단어의 빈도를 이용하여) 각 단어의 고유한 숫자로 표현, 이때 텍스트 간의 유사도

df['Good_Point'], df['Category_Map']를 독립변수, 종속변수로 두고,
기준하게 분류된 df['Category_Map']의 80%로 훈련하고, df['Category_Map']의 80%로 결과를 학습시킨다.


In [64]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(df['Good_Point'], df['Category_Map'], test_size=0.2, random_state=42)

# 텍스트 벡터화
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 분류 모델 학습
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# 예측 및 평가
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print()
print(f"정확도: {accuracy}")

              precision    recall  f1-score   support

     가격 및 가치       1.00      1.00      1.00       241
         서비스       1.00      1.00      1.00       177
    시설 및 분위기       1.00      1.00      1.00      1031
      음식의 속성       1.00      1.00      1.00       637
      주변 인프라       1.00      1.00      1.00        86

    accuracy                           1.00      2172
   macro avg       1.00      1.00      1.00      2172
weighted avg       1.00      1.00      1.00      2172


정확도: 0.9995395948434622


# 명사와 형용사로 토큰화 후 분류 -> 정확도 평가
- 명사와 형용사 또한 리뷰에 따른 속성에 맞게, 새롭게 분류
- 단어로 토큰화된 것을 통해 분류할 때, 97%
- 리뷰 자체로 분리를 했을 때, 정확도가 높은 이유는 단어로 분리하게 되면 단어 간의 관계, 의미적 관계를 고려할 수 없다.
- 하지만 리뷰 자체로 하게된다면 완전 새로운 리뷰가 들어오지 않는 이상 정확한 분류를 할 수 있게 도와주기 때문

In [65]:
df[:1]

,Attraction,Good_Point,Selected_People,Category_Map,Pred_Category_Map
803,채그로,뷰가 좋아요,146,시설 및 분위기,시설 및 분위기


In [67]:
# 정확도 계산
accuracy = accuracy_score(df['Category_Map'], df['Pred_Category_Map'])

print(f"정확도: {accuracy}")

정확도: 0.996500920810313


# 단어 기준으로 분리한 것을 속성에 따라 분류하여 혼동행렬 적용
- 실제 데이터와 예측 데이터를 평가하며

In [70]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
# plt.rcParams['font.family'] = 'Malgun Gothic'
font_path = '/malgun.ttf'

# 혼동 행렬 및 분류 보고서 생성
cm = confusion_matrix(df['Category_Map'], df['Pred_Category_Map'])
cr = classification_report(df['Category_Map'], df['Pred_Category_Map'])
accuracy = accuracy_score(df['Category_Map'], df['Pred_Category_Map'])

# 혼동행렬 생성
print('Confusion Matrix:\n', cm)

# 분류보고서 생성 -> 정확도, 정밀도, 재현율, F1 점수 등을 계산
print('Classification Report:\n', cr)

# 전체 정확도 계산
print('Accuracy:', accuracy)

Confusion Matrix:
 [[1139    0    0    0    0]
 [   0  890    0    0    0]
 [   0    0 5203   37    0]
 [   1    0    0 3170    0]
 [   0    0    0    0  420]]
Classification Report:
               precision    recall  f1-score   support

     가격 및 가치       1.00      1.00      1.00      1139
         서비스       1.00      1.00      1.00       890
    시설 및 분위기       1.00      0.99      1.00      5240
      음식의 속성       0.99      1.00      0.99      3171
      주변 인프라       1.00      1.00      1.00       420

    accuracy                           1.00     10860
   macro avg       1.00      1.00      1.00     10860
weighted avg       1.00      1.00      1.00     10860

Accuracy: 0.996500920810313


In [ ]:
# 혼동 행렬 시각화
# plt.rc("font", family='NanumBarunGothic')
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=pred_classification['Category_Map'].unique(), yticklabels=pred_classification['Pred_Category_Map'].unique())
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix')
# plt.show()